In [8]:
import pandas as pd
from itertools import combinations
sample_df= pd.read_csv('investor_startup.csv')

/var/folders/bz/td29f_kd0kb76lk9cgh2c0cw0000gn/T/ipykernel_77455/3363258254.py:1: DtypeWarning: Columns (32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df= pd.read_csv('df_present.csv')


In [9]:


# Convert announced_on to datetime and extract year
sample_df['announced_on'] = pd.to_datetime(sample_df['announced_on'])
sample_df['year'] = sample_df['announced_on'].dt.year

# Create a new column that combines the category and region
sample_df['category_region'] = sample_df['category_groups_list'] + '-' + sample_df['org_region']

# Drop rows with NaN investor_uuids
sample_df = sample_df.dropna(subset=['investor_uuid'])

rivalry_scores = []

for year in sample_df['year'].unique():
    df_year = sample_df[sample_df['year'] == year]
    category_regions = df_year['category_region'].unique()
    pair_counts = {}
    
    for category_region in category_regions:
        df_category_region = df_year[df_year['category_region'] == category_region]
        investors = df_category_region['investor_uuid'].unique()
        
        for investor in investors:
            df_investor = df_category_region[df_category_region['investor_uuid'] == investor]
            startups = df_investor['org_uuid'].unique()
            
            for other_investor in investors:
                if investor == other_investor:
                    continue
                
                df_other_investor = df_category_region[df_category_region['investor_uuid'] == other_investor]
                other_startups = df_other_investor['org_uuid'].unique()
                
                # Check if the two investors have invested in different startups
                if not set(startups).intersection(set(other_startups)):
                    pair = tuple(sorted((investor, other_investor)))
                    if pair in pair_counts:
                        pair_counts[pair] += 1
                    else:
                        pair_counts[pair] = 1
                        
    df_rivalry_scores_year = pd.DataFrame({'investor1': pair[0], 'investor2': pair[1], 'rivalry_score': count} for pair, count in pair_counts.items())
    df_rivalry_scores_year['year'] = year
    rivalry_scores.append(df_rivalry_scores_year)

df_rivalry_scores = pd.concat(rivalry_scores)
df_rivalry_scores


,investor1,investor2,rivalry_score,year
0,0c867fde-2b9a-df10-fdb9-66b74f355f91,65ee9a90-bb84-32ca-bb0c-59020fae9a20,2.0,2005
1,b08efc27-da40-505a-6f9d-c9e14247bf36,c75bfed3-730f-2875-dfdd-11ef59743eae,2.0,2005
2,b08efc27-da40-505a-6f9d-c9e14247bf36,e2006571-6b7a-e477-002a-f7014f48a7e3,8.0,2005
3,b08efc27-da40-505a-6f9d-c9e14247bf36,fe2d1e8b-f607-3c9f-fad7-98fb8412f77e,6.0,2005
4,b08efc27-da40-505a-6f9d-c9e14247bf36,beadb218-e5fa-2686-bc95-4dfaa9acc2e8,4.0,2005
...,...,...,...,...
313486,2600ff12-d51d-38d0-53c8-e5e24e48a5ca,65bbca2e-bf1c-4e94-ab4a-5cb8ce07dad2,2.0,2018
313487,8fb0f072-6f07-4022-ad2c-880abded4c91,cb2925e9-9f80-41c1-a59f-776b347f3e53,2.0,2018
313488,b8d38291-6018-dd90-83ec-0723756f2949,c12fc3c8-f896-810b-9e1c-1e3af25e100b,2.0,2018
313489,1a410398-3a72-5882-99b8-6318cf594850,b8d38291-6018-dd90-83ec-0723756f2949,2.0,2018


In [10]:
df_rivalry_scores.to_csv('rivalry_scores.csv')